In [ ]:
# The following code is for plotting the distribution of values in the CSV files
import numpy as np
import matplotlib.pyplot as plt

data = np.loadtxt('../data/img1/erfnet_raw_out_0_2332400_2332401_200_1.csv', delimiter=',')
faulty = data[1].astype(int)
free = data[0].astype(int)

unique_vals = np.setdiff1d(faulty, free)
print(unique_vals)
print(free.min())


MIN_VALUE = 85
MAX_VALUE = 127
STEP = 4

bins = np.arange(MIN_VALUE, MAX_VALUE + 2)  # Create bins from 1 to 128 (inclusive of 127)
faulty_counts, _ = np.histogram(faulty, bins=bins)
free_counts, _ = np.histogram(free, bins=bins)

x = np.arange(MIN_VALUE, MAX_VALUE + 1)  # x values for the bars

plt.figure(figsize=(6, 6))
plt.bar(x - 0.2, free_counts, width=0.4, label='Free', color='blue')
plt.bar(x + 0.2, faulty_counts, width=0.4, label='Faulty', color='red')

plt.xlabel('Value (0–127)')
plt.ylabel('Count')
plt.title('Value Distribution')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.xticks(np.arange(MIN_VALUE, MAX_VALUE+1, STEP))
plt.tight_layout()
plt.show()

In [ ]:
# The following code is for visualizing the 3D data using ipywidgets, I used Jupyter Notebook in VScode
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
data = np.loadtxt('../data/model_layers/layer-49/erfnet_raw_out_0_31818_31819_400_1.csv', delimiter=",")
faulty = data[0].astype(int)
faulty_3d = faulty.reshape(128, 256, 64)
vmin, vmax = 0, 127
def show_slice(z):
    plt.figure(figsize=(20, 10))
    plt.imshow(faulty_3d[:, :, z], cmap='Greens', interpolation='none', vmin=vmin, vmax=vmax)
    plt.title(f"Z-slice {z}")
    plt.colorbar(label="Pixel Value")
    plt.show()

widgets.interact(show_slice, z=widgets.IntSlider(min=0, max=127, step=1, value=0))

In [ ]:
# The following is to analyze the 3d values (should be run as a standalone program)
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import sys
from matplotlib import cm
from collections import Counter

# Load CSV data passed as first argument
data = np.loadtxt(sys.argv[1], delimiter=',')

# Expected 3D shape dimensions
d1 = 128
d2 = 256
d3 = 64

# Threshold for difference
th = 0

# Reshape data: ignore first column in each row
faulty = (data[1, 1:]).reshape(d1, d2, d3)
free = (data[0, 1:]).reshape(d1, d2, d3)
num_different = np.sum(faulty != free)
print(num_different)

# Compute absolute difference
diff_f = np.abs(free - faulty)

# Mask where the difference exceeds threshold
mask = diff_f > th
x = np.where(mask)[0]
y = np.where(mask)[1]
z = np.where(mask)[2]

# Choose color values based on user input
if sys.argv[2] == "faulty":
    colo = faulty[mask]
    volume = data[1, 1:]
    array3d = faulty
elif sys.argv[2] == "free":
    colo = free[mask]
    volume = data[0, 1:]
    array3d = free
else:
    raise ValueError("Second argument must be either 'faulty' or 'free'")

# Create plot
fig = plt.figure(figsize=(8.2, 7))
ax = fig.add_subplot(111, projection='3d')

# Create scatter plot with color mapping
img = ax.scatter(x, y, z, marker='o', s=6, c=colo, cmap='Greens')
plt.colorbar(img, ax=ax)

# Set axis limits
ax.set_xlim(0, d1)
ax.set_ylim(0, d2)
ax.set_zlim(0, d3)
plt.show()